In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
import math
import random
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [18]:
data1 = pd.read_csv("../../dataset/NASA/KC3.csv")
data1.head()

,LOC,BRANCHCOUNT,CALLPAIRS,LOCCODEANDCOMMENT,LOCCOMMENTS,CONDITIONCOUNT,CYCLOMATICCOMPLEXITY,CYCLOMATICDENSITY,DECISIONCOUNT,DECISIONDENSITY,...,NODECOUNT,NORMALIZEDCYLOMATICCOMPLEXITY,NUMOPERANDS,NUMOPERATORS,NUMUNIQUEOPERANDS,NUMUNIQUEOPERATORS,NUMBEROFLINES,PERCENTCOMMENTS,LOCTOTAL,Defective
0,12,15,20,1,7,16,9,0.70,8,2.00,...,49,0.12,102,185,54,19,73,13.33,53,Y
1,0,3,4,0,0,4,2,0.25,2,2.00,...,9,0.22,15,27,10,12,9,0.00,8,N
2,3,5,3,0,1,6,3,0.27,2,3.00,...,12,0.19,34,51,15,12,16,8.33,11,N
3,11,43,16,0,4,76,22,0.21,34,2.24,...,99,0.18,208,349,51,24,119,3.74,103,N
4,9,11,13,0,9,8,8,0.10,4,2.00,...,37,0.08,152,215,71,18,102,10.23,79,Y


In [19]:
data1.describe()

,LOC,BRANCHCOUNT,CALLPAIRS,LOCCODEANDCOMMENT,LOCCOMMENTS,CONDITIONCOUNT,CYCLOMATICCOMPLEXITY,CYCLOMATICDENSITY,DECISIONCOUNT,DECISIONDENSITY,...,MULTIPLECONDITIONCOUNT,NODECOUNT,NORMALIZEDCYLOMATICCOMPLEXITY,NUMOPERANDS,NUMOPERATORS,NUMUNIQUEOPERANDS,NUMUNIQUEOPERATORS,NUMBEROFLINES,PERCENTCOMMENTS,LOCTOTAL
count,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,...,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,4.809278,10.835052,9.515464,0.231959,2.463918,14.113402,6.350515,0.233351,6.628866,2.127010,...,7.061856,33.690722,0.182680,69.005155,117.809278,29.314433,16.072165,41.304124,5.660619,32.597938
std,5.785431,10.075035,7.841187,0.847312,4.788938,14.259168,5.623860,0.092901,6.581210,0.320942,...,7.129904,32.163881,0.070344,78.049639,126.712394,25.492437,4.896329,42.817425,8.538215,34.191372
min,0.000000,3.000000,1.000000,0.000000,0.000000,4.000000,2.000000,0.090000,2.000000,2.000000,...,2.000000,6.000000,0.060000,5.000000,13.000000,4.000000,7.000000,5.000000,0.000000,4.000000
25%,1.000000,3.000000,4.000000,0.000000,0.000000,4.000000,2.000000,0.170000,2.000000,2.000000,...,2.000000,13.000000,0.130000,22.000000,41.250000,13.000000,13.000000,14.000000,0.000000,11.250000
50%,3.000000,7.000000,7.000000,0.000000,0.000000,8.000000,4.000000,0.220000,4.000000,2.000000,...,4.000000,22.000000,0.170000,39.000000,69.500000,20.000000,16.000000,25.000000,0.465000,20.000000
75%,7.000000,13.000000,12.000000,0.000000,3.000000,18.000000,7.750000,0.270000,8.000000,2.070000,...,9.000000,42.750000,0.210000,88.500000,151.250000,40.500000,19.000000,55.500000,9.535000,43.000000
max,28.000000,59.000000,46.000000,9.000000,38.000000,76.000000,36.000000,0.700000,34.000000,4.000000,...,38.000000,201.000000,0.430000,556.000000,857.000000,159.000000,31.000000,310.000000,43.750000,242.000000


In [20]:
data1.values.shape

(194, 40)

In [21]:
data2=data1.values
for i in range(194):
    if data2[i][39]=="Y":
        data2[i][39]=1
    else:
        data2[i][39]=0
data2[1]

array([0, 3, 4, 0, 0, 4, 2, 0.25, 2, 2.0, 2, 1.0, 9, 1, 0.0, 8, 2, 2, 1.0,
       20.81, 9.0, 1685.67, 0.06, 42, 0.11, 93.65, 187.3, 0.5, 1, 2, 9,
       0.22, 15, 27, 10, 12, 9, 0.0, 8, 0], dtype=object)

In [22]:
X=np.array(data2).astype(float)  

In [23]:
X[:,:39]

array([[12.  , 15.  , 20.  , ..., 73.  , 13.33, 53.  ],
       [ 0.  ,  3.  ,  4.  , ...,  9.  ,  0.  ,  8.  ],
       [ 3.  ,  5.  ,  3.  , ..., 16.  ,  8.33, 11.  ],
       ...,
       [ 8.  , 16.  , 13.  , ..., 65.  , 16.36, 46.  ],
       [ 0.  ,  7.  ,  5.  , ..., 12.  ,  0.  , 11.  ],
       [ 2.  , 12.  ,  5.  , ..., 25.  ,  4.55, 21.  ]])

In [24]:
y=X[:,39]

In [25]:
X=X[:,0:39]

In [26]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
#os =  RandomOverSampler()
X_res, y_res = SMOTE().fit_resample(X, y)
X_res.shape,y_res.shape

((316, 39), (316,))

In [27]:
X_res.shape
#X_res=X_res.reshape(898,21,1)
#X_res.shape

(316, 39)

In [28]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import datetime
import csv
import time

def show_accuracy(predictLabel,Label):
    Label = np.ravel(Label).tolist()
    predictLabel = predictLabel.tolist()
    count = 0
    for i in range(len(Label)):
        if Label[i] == predictLabel[i]:
            count += 1
    return (round(count/len(Label),5))

class node_generator(object):
    def __init__(self, whiten = False):
        self.Wlist = []
        self.blist = []
        self.function_num = 0
        self.whiten = whiten

    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(x, 0)

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def linear(self, x):
        return x

    def orth(self, W):
        """
        目前看来，这个函数应该配合下一个generator函数是生成权重的
        """
        for i in range(0, W.shape[1]):
            w = np.mat(W[:,i].copy()).T
            w_sum = 0
            for j in range(i):
                wj = np.mat(W[:,j].copy()).T
                w_sum += (w.T.dot(wj))[0,0]*wj
            w -= w_sum
            w = w/np.sqrt(w.T.dot(w))
            W[:,i] = np.ravel(w)

        return W

    def generator(self, shape, times):
        for i in range(times):
            random.seed(i)
            W = 2*np.random.random(size=shape)-1
            if self.whiten == True:
                W = self.orth(W)   # 只在增强层使用
            b = 2*np.random.random() -1
            yield (W, b)

    def generator_nodes(self, data, times, batchsize, function_num):
        # 按照bls的理论，mapping layer是输入乘以不同的权重加上不同的偏差之后得到的
        # 若干组，所以，权重是一个列表，每一个元素可作为权重与输入相乘
        self.Wlist = [elem[0] for elem in self.generator((data.shape[1], batchsize), times)]
        self.blist = [elem[1] for elem in self.generator((data.shape[1], batchsize), times)]

        self.function_num = {'linear':self.linear,
                        'sigmoid': self.sigmoid,
                        'tanh':self.tanh,
                        'relu':self.relu }[function_num]  # 激活函数供不同的层选择
        # 下面就是先得到一组mapping nodes，再不断叠加，得到len(Wlist)组mapping nodes
        nodes = self.function_num(data.dot(self.Wlist[0]) + self.blist[0])
        for i in range(1, len(self.Wlist)):
            nodes = np.column_stack((nodes, self.function_num(data.dot(self.Wlist[i])+self.blist[i])))
        return nodes

    def transform(self,testdata):
        testnodes = self.function_num(testdata.dot(self.Wlist[0])+self.blist[0])
        for i in range(1,len(self.Wlist)):
            testnodes = np.column_stack((testnodes, self.function_num(testdata.dot(self.Wlist[i])+self.blist[i])))
        return testnodes

class scaler:
    def __init__(self):
        self._mean = 0
        self._std = 0
    
    def fit_transform(self,traindata):
        self._mean = traindata.mean(axis = 0)
        self._std = traindata.std(axis = 0)
        return (traindata-self._mean)/(self._std+0.001)
    
    def transform(self,testdata):
        return (testdata-self._mean)/(self._std+0.001)

class broadnet:
    def __init__(self, 
                 maptimes = 10, 
                 enhencetimes = 10,
                 map_function = 'linear',
                 enhence_function = 'linear',
                 batchsize = 'auto', 
                 reg = 0.001):
        
        self._maptimes = maptimes
        self._enhencetimes = enhencetimes
        self._batchsize = batchsize
        self._reg = reg
        self._map_function = map_function
        self._enhence_function = enhence_function
        
        self.W = 0
        self.pesuedoinverse = 0
        self.normalscaler = scaler()
        self.onehotencoder = preprocessing.OneHotEncoder(sparse = False)
        self.mapping_generator = node_generator()
        self.enhence_generator = node_generator(whiten = True)

    def fit(self,data,label,weight):
        if self._batchsize == 'auto':
            self._batchsize = data.shape[1]
        data = self.normalscaler.fit_transform(data)
        label = self.onehotencoder.fit_transform(np.mat(label).T)
        
        mappingdata = self.mapping_generator.generator_nodes(data,self._maptimes,self._batchsize,self._map_function)
        enhencedata = self.enhence_generator.generator_nodes(mappingdata,self._enhencetimes,self._batchsize,self._enhence_function)
        
        #print('number of mapping nodes {0}, number of enhence nodes {1}'.format(mappingdata.shape[1],enhencedata.shape[1]))
        #print('mapping nodes maxvalue {0} minvalue {1} '.format(round(np.max(mappingdata),5),round(np.min(mappingdata),5)))
        #print('enhence nodes maxvalue {0} minvalue {1} '.format(round(np.max(enhencedata),5),round(np.min(enhencedata),5)))
        
        inputdata = np.column_stack((mappingdata,enhencedata))
        pesuedoinverse = self.pinv(inputdata,self._reg,weight)
        self.W =  pesuedoinverse.dot(label)
        
        #print('W:', self.W)
        #print('W:', self.W.shape)  
    
    #改写伪逆矩阵算法，将权重输入
    def pinv(self,A,reg,weight):
        return np.mat(reg*np.eye(A.shape[1])+A.T.dot(weight).dot(A)).I.dot(A.T).dot(weight)
    
    def decode(self,Y_onehot):
        Y = []
        for i in range(Y_onehot.shape[0]):
            lis = np.ravel(Y_onehot[i,:]).tolist()
            Y.append(lis.index(max(lis)))
        return np.array(Y)
    
    def accuracy(self,predictlabel,label):
        label = np.ravel(label).tolist()
        predictlabel = predictlabel.tolist()
        count = 0
        for i in range(len(label)):
            if label[i] == predictlabel[i]:
                count += 1
        return (round(count/len(label),5))
        
    def predict(self,testdata):
        testdata = self.normalscaler.transform(testdata)
        test_mappingdata = self.mapping_generator.transform(testdata)
        test_enhencedata = self.enhence_generator.transform(test_mappingdata)
        
        test_inputdata = np.column_stack((test_mappingdata,test_enhencedata)) 
        #print('*predictlabel shape:',self.decode(test_inputdata.dot(self.W)).shape)
        #print('*predictlabel:', self.decode(test_inputdata.dot(self.W)))
        #print('*accuracy:',show_accuracy(self.decode(test_inputdata.dot(self.W)),testlabel))
        return self.decode(test_inputdata.dot(self.W))
    
    def addWeight(self,trainlabel):
        #对BLS设置加权，按照训练集中的label比例进行权重设置
        #求训练数据集中label为1的个数
        count=0
        for z in range(len(trainlabel)):
            #print ('个数 %d ' %z)
            #print('k_train_label:', k_train_label[z])
            if trainlabel[z]==1:
                count=count+1
        #print ('label为1的个数 %d ' %j)
        #print(count)
        #print(k_train_label.shape)
        #print(len(k_train_label))
        #权重设置 
        weight = np.zeros((len(trainlabel),len(trainlabel)))
        for i in range(len(trainlabel)):
            if trainlabel[i]==1:
                #weight[i,i]=0.618/count
                weight[i,i]=2*(len(trainlabel)-count)/len(trainlabel)
            else:
                #weight[i,i]=1/(len(k_train_label)-count)
                weight[i,i]=2*count/len(trainlabel)
        #print('Weight:', weight)
        return weight

In [29]:
#将数据集划分为训练集和测试集
#traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.2,random_state = 0)
#print(traindata.shape,trainlabel.shape,testdata.shape,testlabel.shape)
#X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
#y_train = np_utils.to_categorical(y_train)
#y_test = np_utils.to_categorical(y_test)

In [30]:
Evaluat_list=[]
for i in np.arange(0,10):
    #将数据集划分为训练集和测试集
    traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.25,random_state = i)
    bls = broadnet(maptimes = 30, 
               enhencetimes = 15,
               map_function = 'relu',
               enhence_function = 'relu',
               batchsize =100,
               reg = 0.001)

    #根据训练数据设置权重
    weight=bls.addWeight(trainlabel)

    #训练
    starttime = datetime.datetime.now()
    bls.fit(traindata,trainlabel,weight)
    endtime = datetime.datetime.now()
    trainingTime=(endtime - starttime).total_seconds()
    #print('the training time of BLS is {0} seconds'.format((endtime - starttime).total_seconds()))

    #预测
    predictlabel = bls.predict(testdata)

    #评价指标计算
    acc=accuracy_score(testlabel,predictlabel,normalize=True)
    fmeasure=f1_score(testlabel,predictlabel, average='weighted', labels=np.unique(testlabel))
    try:
        auc=roc_auc_score(testlabel,predictlabel, average='macro', sample_weight=None)
    except ValueError:
        pass
    MCC=matthews_corrcoef(testlabel,predictlabel)
    Gmeasure=geometric_mean_score(testlabel,predictlabel, average='weighted')
    recall=recall_score(testlabel, predictlabel, average='weighted')

    print('trainingTime：%f,acc：%f,fmeasure：%f,auc：%f,recall：%f,MCC：%f,Gmeasure：%f'%(trainingTime,acc,fmeasure,auc,recall,MCC,Gmeasure))
    Evaluat_tuple=(trainingTime,acc,fmeasure,auc,recall,MCC,Gmeasure)    
    Evaluat_list.append(Evaluat_tuple)

# 表头
header = ['trainTime', 'acc', 'fmeasure', 'auc','recall', 'MCC', 'Gmeasure']
with open('./data_remember/0.75WBLS_Smote_KC3_Evaluat.csv', 'w', encoding='utf-8', newline='') as file_obj:
    # 创建对象
    writer = csv.writer(file_obj)
    # 写表头
    writer.writerow(header)
    # 3.写入数据(一次性写入多行)
    writer.writerows(Evaluat_list)


    #sio.savemat('./data_remember/WBLS_Smote_CM1_test.mat',{'time':format((endtime - starttime).total_seconds()),'acc':acc,'f1':fmeasure,'auc':auc,'mcc':MCC,'Gm':Gmeasure})

trainingTime：3.671087,acc：0.936709,fmeasure：0.936729,auc：0.937099,recall：0.936709,MCC：0.873637,Gmeasure：0.937099
trainingTime：3.646503,acc：0.860759,fmeasure：0.860759,auc：0.862003,recall：0.860759,MCC：0.724005,Gmeasure：0.862002
trainingTime：3.647859,acc：0.898734,fmeasure：0.899650,auc：0.914894,recall：0.898734,MCC：0.814758,Gmeasure：0.914751
trainingTime：3.669102,acc：0.860759,fmeasure：0.858265,auc：0.862500,recall：0.860759,MCC：0.751997,Gmeasure：0.862498
trainingTime：3.667965,acc：0.873418,fmeasure：0.871987,auc：0.878049,recall：0.873418,MCC：0.773678,Gmeasure：0.878037
trainingTime：3.666090,acc：0.898734,fmeasure：0.897551,auc：0.897436,recall：0.898734,MCC：0.813875,Gmeasure：0.897435
trainingTime：3.649933,acc：0.924051,fmeasure：0.924026,auc：0.924359,recall：0.924051,MCC：0.849263,Gmeasure：0.924359
trainingTime：3.559687,acc：0.797468,fmeasure：0.796165,auc：0.801027,recall：0.797468,MCC：0.609935,Gmeasure：0.801019
trainingTime：3.554079,acc：0.911392,fmeasure：0.910382,auc：0.907895,recall：0.911392,MCC：0.834758,G